# Identification of all lncRNAs in Arabidopsis
### By Group 1 : Jiang, Zac, Xianlin, and Alaina 
### Formatting, integration credit: Zac


## Step 0.1 - import modules; define file input, output locations
### Credit: all

In [87]:
from Bio import SeqIO
import pandas as pd
import csv
import os
import myGTF
from operator import itemgetter 
from Bio.SeqIO.FastaIO import SimpleFastaParser
from Bio.Data import CodonTable
import operator

#WorkDir = input() # for custom folder input
#WorkDir = "/groups/ALS5224/Group1/PythonProject/"
WorkDir = "/home/zacjohnson/OneDrive/School-dj-Ubuntu/DataScience/ProjectDataTest/"
os.chdir(WorkDir)

# Input
GTF = WorkDir + "GTF.gtf"
RawSequence = WorkDir + "Ath.fa"

# Intermediates
TranscriptSeq = WorkDir + "transcriptSeq.fasta"
TranslatedORF = WorkDir + "translatedORF.fasta"
LongestORF = WorkDir + "longestORF.fasta"

# Output
lncRNAcsv = WorkDir + "lncRNA.csv"

### Step 0.2 - Import GTF file, parse raw FASTA sequence for all exons
#### Output: transcriptSeq.fa
#### Credit: Song Li, modifications by Zac and Jiang

In [88]:
## Import GTF file, create TS start/stop positions
ts2pos = {}   
ts2chr = {}
for k in myGTF.lines('large.gtf'): 
    tsid = k['transcript_id']; chrname = k['seqname']
    tsstart = int(k['start']); tsend = int(k['end'])

    if not tsid in ts2pos: 
        ts2pos[tsid]=[]    
        ts2pos[tsid].append([tsstart,tsend]) 
    else:
        ts2pos[tsid].append([tsstart,tsend])

    if not tsid in ts2chr:
        ts2chr[tsid]=chrname

# Create sorted dictionary of start/stop positions
ts2pos_s = {}
for ets in ts2pos:
    ts2pos_s[ets]=sorted(ts2pos[ets], key = itemgetter(1)) 
    
# Parse raw sequence file for start/stop positions
chr2seq = {}
for seq_record in SeqIO.parse(RawSequence, "fasta"):
    chr2seq[seq_record.id] = seq_record.seq 
    print(seq_record.id, len(seq_record))

# Write exons in raw sequence file to new dictionary    
ts2seq = {}
for eachts in ts2chr:
    tschr = ts2chr[eachts]        
    exonpos = ts2pos_s[eachts]
    chrseq = chr2seq[tschr]
    tmpseq = ''
    for eachexon in exonpos:
        exonstart = eachexon[0]
        exonend = eachexon[1]
        exonseq = chrseq[exonstart-1:exonend] 
        if tmpseq == '':               
            tmpseq = exonseq           
        else:                          
            tmpseq = tmpseq + exonseq  
    ts2seq[eachts] = tmpseq

# Write exons to new file with fasta formatting
with open("transcriptSeq.fasta", "w") as ofile:
    for i in ts2seq:
        ofile.write(">" + i + "\n" +str(ts2seq[i]) + "\n")
ofile.close()

print("Step 0 finished")

Pt 154478
Mt 366924
4 18585056
2 19698289
3 23459830
5 26975502
1 30427671
Step 0 finished


### Step 1 - Define ORF locations as nucleotide fasta
#### Output: translateORF.fa
#### Credit: https://github.com/zkstewart, implemented by Jiang

In [89]:
# to run a terminal command to use biopython_orf_find.py in chdir
# Removed 'True' tag, was giving error
#to run a terminal command to use biopython_orf_find.py, result saved in 'translatedORF.fa'
#the biopython_orf_find.py is modified from https://github.com/zkstewart/orf-finder-py
#the modification is that I removed the ORF without stop codons(line 368 to line371)
#-min is the minimum length of the found protein
#-max is the maximum length of the found protein
#-st is the format of output(nucleotide/protein)
# NUCL = nucleotide, PROT = Protein, BOTH = both
#-num is the number of protein translated per transcript(if multiple, sorted from long to short, if 1, only the longest)
#-f is force write/replace file with same name
#-n doesn't change the fasta header

os.system('python3 biopython_orf_find.py -i transcriptSeq.fasta  -o longestORF.fasta -min 1 -max 10000 -st NUCL -num 1 -f Y -n')
print("Step 1 finished")

Step 1 finished


### Step 2 - Create Protein FASTA file from Gene FASTA file
#### Credit Xianlin

In [91]:

#Use standard table to translate gene sequence into protein sequence
from Bio.Data import CodonTable
standard_table = CodonTable.unambiguous_dna_by_name["Standard"]

Geneseq = {} #create an empty directory
for record in SeqIO.parse(LongestORF,"fasta"):
    Geneseq[record.id] = record.seq.translate(standard_table)

proseq = open(TranslatedORF, "w")
for i in Geneseq:
    proseq.write(">"+i+'\n'+str(Geneseq[i]).rstrip("*")+'\n')
proseq.close()

print("Step 2 finished")

Step 2 finished


### Step 3 - Create transcript list from Protein Fasta file
#### Credit: Jiang

In [92]:
orfDic = {}
transcriptDic = {}
longer_than_200 = []
orf_shorter_than_100 = []
prolen = []

# Find all ORFs > 200 nucleotides long
for record in SeqIO.parse(TranscriptSeq,'fasta'):
    transcriptDic[record.id] = record.seq
    if len(record.seq) > 200:
        longer_than_200.append(record.id)
        
# Find all ORFs < 100aa long
for record in SeqIO.parse(TranslatedORF,'fasta'):
    orfDic[record.id] = record.seq
    if len(record.seq) < 100:
        orf_shorter_than_100.append(record.id)

orfKeyList = orfDic.keys()
transcriptKeyList = transcriptDic.keys()

## lncRNA is longer than 200nt, has no orf or orf is shorter than 100 aa
seqNoOrf = set(transcriptKeyList) - set(orfKeyList)
lncRNA = set(longer_than_200) & set(orf_shorter_than_100) | seqNoOrf 

#get protein length, if no orf, protein length is set to 0
for i in transcriptKeyList:
    if i in orfKeyList:
        prolen.append(len(orfDic[i]))
    else:
        prolen.append(0)

lncrna_bool = []
for i in transcriptKeyList:
    if i in lncRNA:
        lncrna_bool.append('YES')
    else:
        lncrna_bool.append('NO')

print("Step 3 finished")

Step 3 finished


### Step 4 - Find lncRNAs and length of longest ORF < 100 amino acids
#### Credit: Alaina and Zac

In [93]:
## Getting length of ORFs - Alaina
FastaFile = open(TranslatedORF, 'r')
LenFile = open('gene_lengths.csv', 'w')

for name, seq in SimpleFastaParser(FastaFile):
    seqLen = len(seq)
    LenFile.write(name + ',' + str(seqLen) + '\n')

FastaFile.close()
LenFile.close()

columns_title = ['TranscriptID', 'ProLen']
gene2length = {}
output1 = open("gene_100aa_less.txt","w")
output2 = open("gene_100aa_less_sorted.txt","w")
with open("gene_lengths.csv", 'r') as input1: 

    for i in input1.readlines():
        line = i.rstrip()
        fields = line.split(",")
        gene = fields[0]
        length = int(fields[1])
        gene2length[gene] = length
        if gene2length[gene] < 100:
            keep = [gene, str(gene2length[gene])]
            keep_gene = gene
            keep_length = str(gene2length[gene])
            output1.write(keep_gene + "\t" + keep_length + "\n")

with open("gene_100aa_less.txt","r") as input2:
    sorting_table = pd.read_table(input2, header = None)
    lengths_sorted = sorting_table.sort_values(1)
    print(lengths_sorted.head())
    print(lengths_sorted.nlargest(10,1))
    lengths_csv = lengths_sorted.to_csv(header = None)
    output2.write(lengths_csv)
input1.close()
input2.close()

print("Step 4 finished")

                  0  1
438  TCONS_00004068  2
568  TCONS_00005562  2
677  TCONS_00007067  2
683  TCONS_00007097  2
15   TCONS_00000199  2
                  0   1
787  TCONS_00008078  99
830  TCONS_00008283  99
344  TCONS_00003257  99
748  TCONS_00007920  99
65   TCONS_00000842  99
522  TCONS_00005026  99
29   TCONS_00000429  99
511  TCONS_00004797  99
512  TCONS_00004798  99
691  TCONS_00007156  99
Step 4 finished


### Step 4.1 Define lncRNAs
#### Credit Jiang and Zac

In [94]:
## Define TCs that are lncRNA 
columns_title_lncRNA = ['TranscriptID', 'ProLen', 'IslncRNA']
data = list(zip(transcriptKeyList, prolen, lncrna_bool))
df = pd.DataFrame(data, columns=columns_title_lncRNA) # Create dataframe of all transcripts
Data_YesOnly = df.loc[df.IslncRNA == "YES"]


with open(lncRNAcsv, "w") as lnc: # Write lncRNA transcripts only to csv
    Data_YesOnly.to_csv(lnc)
lncRNA_Only.close()
print(Data_YesOnly.head())


lncRNAlen = {}
for i in transcriptDic:
   if i in lncRNA:
       lncRNAlen[i] = len(transcriptDic[i])
lncMAX = max(lncRNAlen.items(), key=operator.itemgetter(1))[0]

print("The following transcript is the longest lncRNA: ")
print(lncMAX,transcriptDic[lncMAX],lncRNAlen[lncMAX])
    

      TranscriptID  ProLen IslncRNA
3   TCONS_00000004      22      YES
32  TCONS_00000033      83      YES
34  TCONS_00000035      77      YES
38  TCONS_00000039      71      YES
68  TCONS_00000069       0      YES
The following transcript is the longest lncRNA: 
TCONS_00003043 TGAATTCCTTTTCATCTTTATTTTTTAGAATGGTCCAACTCCAACCAAGTCATTTCTTTTTAAACAATCTTTGTTTTGCTTTTTGAAAATTCGTCCATAGACTCACTGTCATGGGTTAATATAAGTTGTATGTGCAAAATTACTATACTCTAATATTTACTTAGGCAATAATATGATAAATCTCTGAAAATACTCTTTTGGTTCGTGAACATTTGGTATTTTTTTTATAATAATGTTATTGTGTAAATTAGCCTGGAACAATAATTATATTTGTTTACTAGTTATATTAGACAAAATAACCTGTTATCGAAAACAAAATTTCTCCCAATGGTCAACATATCCTTGCGATGGACGAAATGATGTCAAACAGTTTTTTCGAATAATTTCATATAATATTCAAAAAAAAAAGAAGTTGTTTTTTCCAATCAGGTAGTCTTAAACAAATCTGCAACATAAGTTATATGTTTTAAAAAATATACTCTATGGTTATTTGGGTTATATTCATCAATCGGTAGGTCAGTCCATGACAACGTACTCTAAATTATTAGGAGCAAAATCATGATCTAGTTTAATAAACGGTCTAAACAGACTAATTTTTTTTTTCTGGGAATAAGATAGGGTTAGACCACATGGATAATATGCATGTCCAAATTCTTTAAAATCTAAGCCTTAAAATTTGTTGATAATCAGAATTCTTTTCTATGTCTCTTGTAGAACAAA